In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Generate import Generate
from Traversal_Glove import Meta_Poetry_Glove

from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors

from functions import *
from model_back import Model as Model_back

import os
from six.moves import cPickle

In [3]:
poetic_vectors = KeyedVectors.load_word2vec_format('poetic_embeddings.300d.txt', binary=False)
#glove = KeyedVectors.load_word2vec_format("/Users/chris/Downloads/glove.6B/glove.6B.300d.w2v.txt")

In [4]:
g = Generate(wv=poetic_vectors, save_dir_back="combined_back")

In [5]:
mp = Meta_Poetry_Glove(wv=poetic_vectors)

# Experiments

In [5]:
import tensorflow as tf
from model_forw import Model as Model_forw
from model_back import Model as Model_back
import os
from six.moves import cPickle

In [6]:
def force_middle(model, vocab, prob_sequence, sequence, sess, lst, next_word):
    def decayRepeat(word,sequence, scale):
        safe_repeat_words = []
        #safe_repeat_words = set(["with,the,of,in,i"])
        score_adjust = 0
        decr = -scale
        for w in range(len(sequence)):
            if(word==sequence[w] and word not in safe_repeat_words):
                score_adjust += decr
            decr += scale/10 #decreases penalty as the words keep getting further from the new word
        return score_adjust

    poem = []
    line_num = 0
    wordPool_ind = 0
    ret=[]
    for tup in lst:
        state_init = tup[1][0][1]
        prob_seq =  tup[1][0][0]
        sequence=tup[1][1]
        scale = .02 #scale is the significant magnitude required to affect the score of bad/good things
        dist, state = model.compute_fx(sess, vocab, prob_seq, sequence, state_init, 1)
        #PREVENTS REPEAT ADJACENT WORDS OR PROBLEM-TAGGED WORDS
        if(next_word == sequence[-1]):
            continue
        #if(partsOfSpeechFilter(sequence[-1],next_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
        #    continue
        #FACTORS IN SCORE ADJUSTMENTS
        score_adjust = decayRepeat(next_word, sequence, 100*scale) #repeats
        score_adjust += scale*len(next_word)/50 #length word
        #if(next_word in wordPool):
        #    score_adjust += scale
        #CALCULATES ACTUAL SCORE
        key = np.array([[vocab[next_word]]])
        new_prob = dist[key]
        score_tuple = (new_prob, state)
        score_tup = (score_tuple[0]+score_adjust,score_tuple[1]) #NOTE SHOULD SCORE_ADJUST BE ADDED HERE OR JUST IN THE ITEM LINE?
        item = (score_tup[0],(score_tup, sequence+[next_word]))
        if(item[0]==[[-float("inf")]]):
            continue
        ret+=[item]
    ret.sort(key=itemgetter(0), reverse = True)
    return ret[:g.get_width()]


In [7]:
def generate_line_collocations(word1, word2, collocations):
    # check for collocations and fill in
    postag= g.pos_synset([word1, word2], g.get_postag_dict()[0])

    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]

    line = ['' for i in range(len(template))]
    line[min(postag[1])] = first_noun
    line[max(postag[1])] = last_noun
    
    line = g.insert_collocations(template, line, collocations)
    
    # finding indices with words in them
    not_blank = []
    pairs = []
    for i in range(len(line)):
        if line[i] is not '':
            pairs.append(i)
        if len(pairs) > 1:
            not_blank.append((pairs[0], pairs[1]))
            pairs.pop(0)
    # if haven't made it to the end of the sentence
    if pairs[0] != len(line) - 1:
        not_blank.append((pairs[0], len(line) - 1))

    # iterating over pairs of filled in words, filling in between
    candidates = []

    first_time = True
    first_noun = line[not_blank[0][0]]
    first_pos = not_blank[0][0]
    for i, j in not_blank:
        last_noun = line[j]
        last_pos = j
        
        ### Matias's code
        
        if first_time:
            list_1= g.genPoem_forward(first_noun, template[first_pos:last_pos])
            list_2= g.force_middle(list_1[:50], last_noun)
            candidates.extend(list_2)
            first_time = False
            continue

        tf.reset_default_graph()
        with open(os.path.join(g.get_save_dir(), 'config.pkl'), 'rb') as f:
            saved_args = cPickle.load(f)
        with open(os.path.join(g.get_save_dir(), 'words_vocab.pkl'), 'rb') as f:
            word_keys, vocab = cPickle.load(f)
        model = Model_forw(saved_args, True)
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(g.get_save_dir())
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                poem = []
                line_num = 0
                wordPool_ind = 0
                tt_3=[]
                for element in candidates:
                    seq=element[1][1]
                    state = element[1][0][1]
                    init_score = element[1][0][0]
                    seq=element[1][1]
                    lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                      g.get_dict_tag(),g.get_dict_pos(),g.get_width(),
                                         g.get_wordPools()[wordPool_ind], g.get_pos_set(), template[first_pos:last_pos])
                    # case where are generating between
                    # two words and need to force last one
                    if last_noun is not '':
                        lst = force_middle(model, vocab, init_score, seq, sess, lst[:50], last_noun)
                    tt_3+=lst
                tt_3.sort(key=itemgetter(0), reverse = True)
                candidates = tt_3[:50]
    pre_backwards = candidates
        
    # Filling in reverse
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in candidates[:30]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              g.get_dict_tag(), g.get_dict_pos(),g.get_width(),
                                  g.get_wordPools()[wordPool_ind], g.get_pos_set(), template)

                tt_4+=lst
        else:
            print(ckpt)
    tt_4.sort(key=itemgetter(0), reverse = True)
    candidates = tt_4

    return postag, not_blank, line, candidates, pre_backwards

In [18]:
# %%time

# poets = {'Frost':'Grammar/Robert_Frost.txt'}
# filters = [("NN", "NN"), ("JJ", "NN"), ("JJ", "JJ", "NN"), ("JJ", "NN", "NN"), ("NN", "JJ", "NN"), ("NN", "NN", "NN"), ("NN", "PRP", "NN")]
# colls_dict = build_get_collocation_dict(poets, filters=filters)

# with open('colls_dict.p', 'wb') as f:
#     cPickle.dump(colls_dict, f)

In [12]:
with open('colls_dict.p', 'rb') as f:
    colls_dict = cPickle.load(f)

In [13]:
colls_dict['clock']

[[('clock', 'i'), -4.0, 1.0],
 [('clock', 'think'), -4.0, 0.0],
 [('clock', 'hear'), -2.0, 0.0],
 [('clock', 'rattling.'), 3.0, 0.0],
 [('clock', 'maybe/'), 1.0, 0.0],
 [('clock', "shelf's"), -3.0, 0.0],
 [('clock', 'vase'), 2.0, 0.0],
 [('clock', 'picture,'), 4.0, 0.0],
 [('clock', 'luminary'), -1.0, 0.0],
 [('clock', 'sky'), 3.0, 0.0]]

In [14]:
colls_dict['time']

[[('time', 'full'), -3.0, 0.0],
 [('time', 'many'), -2.0, 0.0],
 [('time', 'doubt'), 3.0, 0.0],
 [('time', 'river'), -3.0, 0.0],
 [('time', 'i'), 1.0, 0.0],
 [('time', 'fellow'), 4.0, 0.0],
 [('time', 'dollar'), -3.0, 0.0],
 [('time', 'grow'), 5.0, 0.0],
 [('time', 'saw'), 3.0, 0.0],
 [('time', "'it's"), -1.0, 0.0],
 [('time', 'turn'), 4.0, 0.0],
 [('time', "i'm"), 3.0, 0.0],
 [('time', 'sort'), -4.0, 0.0],
 [('time', 'swear'), -2.0, 0.0],
 [('time', 'awful!'), 3.0, 0.0],
 [('time', 'think'), 2.0, 0.0],
 [('time', 'something'), 4.0, 0.0],
 [('time', 'child'), -4.0, 0.0],
 [('time', "night?'"), 2.0, 0.0],
 [('time', 'growth,'), 3.0, 0.0],
 [('time', 'talk'), 2.0, 0.0],
 [('time', 'talk.'), 2.0, 0.0],
 [('time', 'cow'), -3.0, 0.0],
 [('time', 'apple'), -1.0, 0.0],
 [('time', 'flight:'), 5.0, 0.0],
 [('time', 'spread'), -5.0, 0.0],
 [('time', 'coal-black'), -4.0, 0.0],
 [('time', 'particular'), -1.0, 0.0],
 [('time', "o'clock,"), 1.0, 0.0],
 [('time', 'thousandth'), -1.0, 0.0],
 [('time',

In [15]:
line = generate_line_collocations('time', 'clock', colls_dict)

['time', 'clock']
['NN', 'NN']
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
[(array([[0]]), ((array([[0]]), (LSTMStateTuple(c=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model/model.ckpt-227699


/Users/chris/projects/poetix/model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


In [16]:
line[1]

[(4, 5), (5, 6)]

In [17]:
line[2]

['', '', '', '', 'clock', 'time', '']

In [18]:
line[0]

[['VBG', 'IN', 'IN', 'DT', 'NN', 'NN', ','], [5, 4], ['time', 'clock']]

In [19]:
for i in range(len(line[3])):
    print(' {:0.3}: '.format(line[3][i][0][0][0]), end='')
    print(' '.join(line[3][i][1][1]))

 -46.7: bough on the snow mowing clock time

 -49.5: afraid in the apple wing clock time

 -49.7: afraid in an apple wing clock time

 -50.9: bough in the snow mowing clock time

 -51.2: afraid on the apple wing clock time

 -51.5: afraid in the kitchen wing clock time

 -51.5: bough against the snow mowing clock time

 -51.7: afraid in the thick wing clock time

 -51.8: afraid in this apple wing clock time

 -51.9: afraid against the snow mowing clock time

 -52.3: afraid in the light mowing clock time

 -53.0: afraid in the snow mowing clock time

 -53.2: bough in the kitchen wing clock time

 -53.3: afraid about the snow mowing clock time

 -53.6: past with the thick wing clock time

 -53.6: afraid on the thick wing clock time

 -53.8: afraid in a apple wing clock time

 -53.9: past in the kitchen wing clock time

 -53.9: bough in the thick wing clock time

 -53.9: afraid on the snow mowing clock time



# Using nltk's collocations

In [6]:
from nltk.collocations import *
from nltk.corpus import stopwords
import string
from nltk.metrics import BigramAssocMeasures
import codecs

In [67]:
poets = ['endymion', 'frost', 'modern_poetry', 'whitman', 'wilde', 'ya', 'ya_all']

stop = set(stopwords.words('english'))
punct = str.maketrans('', '', string.punctuation)
tokens=[]

for poet in poets:
    print(poet)
    f = codecs.open(os.path.join('data', poet, 'input.txt'), 'r', encoding='utf-8', errors='replace')
    raw = f.read()
    f.close
    raw = raw.translate(punct).lower()
    tokens.extend(nltk.word_tokenize(raw))

endymion
frost
modern_poetry
whitman
wilde
ya
ya_all


In [68]:
finder = BigramCollocationFinder.from_words(tokens, window_size=3)

In [69]:
finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)

[('the', 'the'),
 ('the', 'of'),
 ('had', 'been'),
 ('i', 'am'),
 ('more', 'than'),
 ('in', 'the'),
 ('a', 'the'),
 ('of', 'the'),
 ('the', 'a'),
 ('i', 'say')]

In [73]:
colls = finder.score_ngrams(BigramAssocMeasures.likelihood_ratio)
filters = [("NN", "NN"), ("JJ", "NN")]

coll_dict = {}
for coll, score in colls:
    if score < 5:
        continue
    
    if len(g.get_postag_dict()[2][coll[0]]) == 0 or len(g.get_postag_dict()[2][coll[1]]) == 0:
        continue

    pos = (g.get_postag_dict()[2][coll[0]][0], g.get_postag_dict()[2][coll[1]][0])
    
    # POS heuristic filter
    if pos not in filters:
        continue
    
    if coll[0] not in coll_dict:
        coll_dict[coll[0]] = []

    coll_dict[coll[0]].append((coll[1], score))

for value in coll_dict.values():
    value.sort(key=itemgetter(1), reverse=True)
coll_dict

{'first': [('time', 333.92606018149223),
  ('day', 37.291406792120874),
  ('impulse', 36.604990307949755),
  ('thing', 36.063309691993666),
  ('step', 21.129448496108246),
  ('meeting', 17.163701667604347),
  ('year', 13.52824435287113),
  ('glance', 13.038910164047515),
  ('line', 12.459724712840453),
  ('gloveless', 11.435044717932533),
  ('haycock', 11.435044717932533),
  ('letter', 10.418750631869091),
  ('ray', 10.255005513416657),
  ('gift', 9.886347939132023),
  ('motor', 9.712242589362534),
  ('propensity', 9.712242589362534),
  ('kite', 9.20854466635326),
  ('outcry', 9.20854466635326),
  ('visit', 7.37708882736959),
  ('slab', 7.174806491416104),
  ('car', 7.009792683358835),
  ('night', 6.752969445189985),
  ('breath', 6.7439160560507245),
  ('intent', 6.6258962796485275),
  ('place', 6.572937381975494),
  ('page', 6.537417351740823),
  ('suggestion', 6.104611177315627),
  ('stung', 5.929977175833011),
  ('street', 5.217710914826366),
  ('choice', 5.173847150354977)],
 'youn

In [75]:
# Assumes sorted by score
def insert_collocations(template, line, collocations, window_width=3):
    for i, w in enumerate(line):
        if w not in collocations:
            continue;

        grams = collocations[w]
        # iterating over all collocations
        for gram in grams:
            col_word = gram[0]

            if len(g.get_postag_dict()[2][col_word]) == 0:
                continue

            gram_pos = g.get_postag_dict()[2][col_word]

            # empty spot and pos matches
            for j in range(i+1, len(line)):
                if line[j] is '' and template[j] in gram_pos and j - i <= window_width:
                    line[j] = col_word
                    break
            # Break out of second loop if broke from third loop
            else:
                continue
            break
    return line

insert_collocations(['PRP', 'VBP', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN'], 
                    ['I', '', '', '', '', 'open', '', '', '', ''], coll_dict)

['I', '', '', '', '', 'open', 'air', '', '', 'water']

In [76]:
def generate_line_nltk_collocations(word1, word2, collocations):
    # check for collocations and fill in
    postag= g.pos_synset([word1, word2], g.get_postag_dict()[0])

    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]

    line = ['' for i in range(len(template))]
    line[min(postag[1])] = first_noun
    line[max(postag[1])] = last_noun
    
    line = insert_collocations(template, line, collocations)
    
    # finding indices with words in them
    not_blank = []
    pairs = []
    for i in range(len(line)):
        if line[i] is not '':
            pairs.append(i)
        if len(pairs) > 1:
            not_blank.append((pairs[0], pairs[1]))
            pairs.pop(0)
    # if haven't made it to the end of the sentence
    if pairs[0] != len(line) - 1:
        not_blank.append((pairs[0], len(line) - 1))

    # iterating over pairs of filled in words, filling in between
    candidates = []

    first_time = True
    first_noun = line[not_blank[0][0]]
    first_pos = not_blank[0][0]
    for i, j in not_blank:
        last_noun = line[j]
        last_pos = j
        
        ### Matias's code
        
        if first_time:
            list_1= g.genPoem_forward(first_noun, template[first_pos:last_pos])
            list_2= g.force_middle(list_1[:50], last_noun)
            candidates.extend(list_2)
            first_time = False
            continue

        tf.reset_default_graph()
        with open(os.path.join(g.get_save_dir(), 'config.pkl'), 'rb') as f:
            saved_args = cPickle.load(f)
        with open(os.path.join(g.get_save_dir(), 'words_vocab.pkl'), 'rb') as f:
            word_keys, vocab = cPickle.load(f)
        model = Model_forw(saved_args, True)
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(g.get_save_dir())
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                poem = []
                line_num = 0
                wordPool_ind = 0
                tt_3=[]
                for element in candidates:
                    seq=element[1][1]
                    state = element[1][0][1]
                    init_score = element[1][0][0]
                    seq=element[1][1]
                    lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                      g.get_dict_tag(),g.get_dict_pos(),g.get_width(),
                                         g.get_wordPools()[wordPool_ind], g.get_pos_set(), template[first_pos:last_pos])
                    # case where are generating between
                    # two words and need to force last one
                    if last_noun is not '':
                        lst = force_middle(model, vocab, init_score, seq, sess, lst[:50], last_noun)
                    tt_3+=lst
                tt_3.sort(key=itemgetter(0), reverse = True)
                candidates = tt_3[:50]
    pre_backwards = candidates
        
    # Filling in reverse
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in candidates[:30]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              g.get_dict_tag(), g.get_dict_pos(),g.get_width(),
                                  g.get_wordPools()[wordPool_ind], g.get_pos_set(), template)

                tt_4+=lst
        else:
            print(ckpt)
    tt_4.sort(key=itemgetter(0), reverse = True)
    candidates = tt_4

    return postag, not_blank, line, candidates, pre_backwards

In [77]:
line = generate_line_nltk_collocations('time', 'clock', coll_dict)

['time', 'clock']
['NN', 'NN']
6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
[(array([[0]]), ((array([[0]]), (LSTMStateTuple(c=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model/model.ckpt-227699


/Users/chris/projects/poetix/model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


In [78]:
for i in range(len(line[3])):
    print(' {:0.3}: '.format(line[3][i][0][0][0]), end='')
    print(' '.join(line[3][i][1][1]))

 -69.4: living eyes have ago while clock time
 -70.3: something eyes have ago while clock time
 -70.3: something years have ago wing clock time
 -70.7: moving years have ago wing clock time
 -70.7: living leaves have ago while clock time
 -70.8: living wheels have ago wing clock time
 -71.4: misting leaves have ago while clock time
 -71.4: living years have ago wing clock time
 -71.5: being eyes have ago while clock time
 -71.7: breathing trees have ago while clock time
 -71.8: something wheels have ago while clock time
 -71.9: moving leaves have ago while clock time
 -72.3: moving trees have ago while clock time
 -72.4: passing eyes have ago while clock time
 -72.4: blowing trees have ago while clock time
 -72.5: saying eyes have ago while clock time
 -72.5: saving years have ago wing clock time
 -72.7: keeping wheels have ago wing clock time
 -72.8: gathering trees have awhile while clock time
 -72.8: coming years have ago wing clock time


# Rhyming WordNet Traversal

In [6]:
import tensorflow as tf
from model_forw import Model as Model_forw

import random
from itertools import permutations

import requests

ps = nltk.stem.PorterStemmer()

In [7]:
# word1 -> (rhyme)
#    \     /
#     word3

mp.traverse_wn_glove('gallery')

('art', 'a room or series of rooms where works of art are exhibited')

In [8]:
params ={'rel_rhy': 'salary'}
response = requests.get('https://api.datamuse.com/words', params=params)

In [9]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import wordnet as wn

def two_word_link(w1, w2, seen_words):
    max_sim = -1
    best_word = None
    best_word_def = None
    
    word_set = set()
    
    for synset in wn.synsets(w1):
        clean_def = remove_stopwords(mp.punct.sub('', synset.definition()))
        word_set.update(clean_def.lower().split())
    for synset in wn.synsets(w2):
        clean_def = remove_stopwords(mp.punct.sub('', synset.definition()))
        word_set.update(clean_def.lower().split())
        
    for other_word in word_set:
        sim = mp.get_glove_sim(w1, other_word)
        sim += mp.get_glove_sim(w2, other_word)
        
        if sim > max_sim and other_word != w1 and other_word != w2 and ps.stem(other_word) not in seen_words:
            max_sim = sim
            best_word = other_word
            
    return best_word

In [10]:
two_word_link('salary', 'gallery', set())

/Users/chris/projects/poetix/Traversal_Glove.py:44: RuntimeWarning: invalid value encountered in true_divide
  avg_w2 /= np.sqrt(np.sum(avg_w2 ** 2))


'art'

In [52]:
def get_five_words(w2, pos_to_words):
    # three connection words
    w_response = requests.get('https://api.datamuse.com/words', params={'rel_rhy': w2}).json()
    rhyme_nnp = set(d['word'] for d in w_response).intersection(pos_to_words['NNP'])
    
#     if len(w_response) < 2 or len(rhyme_nnp) < 1:
#         raise ValueError('Cannot generate limkerick using ', w2)
        
    w1 = w3 = w5 = None
    seen_words = set([ps.stem(w2)])
    
    for r in w_response:
        if r['word'] in g.get_postag_dict()[2] and ps.stem(r['word']) not in seen_words:
            w5 = r['word']
            seen_words.add(ps.stem(w5))
            break
#     w5 = w_response[0]['word']

    w4 = two_word_link(w2, w5, seen_words)
    seen_words.add(ps.stem(w4))

    for r in rhyme_nnp:
        if r in g.get_postag_dict()[2] and ps.stem(r) not in seen_words:
            w1 = r
            seen_words.add(ps.stem(w1))
            break

    w3_response = requests.get('https://api.datamuse.com/words', params={'rel_rhy': w4}).json()
    
#     if len(w3_response) < 1:
#         raise ValueError('Cannot generate limkerick using ', w2)
        
    max_sim = 0
    for r in w3_response:
        if r['word'] not in g.get_postag_dict()[2]:
            continue
        this_sim = mp.get_glove_sim(r['word'], w4)
        if this_sim  > max_sim and ps.stem(r['word']) not in seen_words:
            w3 = r['word']
            max_sim = this_sim
            
    if w5 is None or w3 is None or w1 is None:
        raise ValueError('Cannot generate limerick using ', w2)
    
    seen_words.add(ps.stem(w3))
            
#     w3 = w3_response[0]['word']
    return w1, w2, w3, w4, w5

In [53]:
# print(get_five_words('salary', g.get_postag_dict()[1]))
# print(get_five_words('chicken', g.get_postag_dict()[1]))
# print(get_five_words('battery', g.get_postag_dict()[1]))
# print(get_five_words('phone', g.get_postag_dict()[1]))
print(get_five_words('tree', g.get_postag_dict()[1]))
print(get_five_words('war', g.get_postag_dict()[1]))
print(get_five_words('car', g.get_postag_dict()[1]))

('pre', 'tree', 'lifetime', 'time', 'be')
('centaur', 'war', 'extend', 'end', 'for')


ValueError: ('Cannot generate limerick using ', 'car')

In [13]:
j = 0
with open("./cmudict-0.7b.txt") as f:
    lines = [line.rstrip("\n").split() for line in f if (";;;" not in line)]
    dictMeters = {}
    for i in range(len(lines)):
        line = lines[i]
        newLine = [line[0].lower()]
        if("(" in newLine[0] and ")" in newLine[0]):
            newLine[0] = newLine[0][:-3]
        chars = ""
        for word in line[1:]:
            for ch in word:
                if(ch in "012"):
                    if(ch == "2"):
                        chars+="1"
                    else:
                        chars+=ch
        newLine+=[chars]
        lines[i] = newLine
        if(newLine[0] not in dictMeters): #THIS IF STATEMENT ALLOWS FOR MULTIPLE PRONUNCIATIONS OF A WORD
            dictMeters[newLine[0]]=[chars]
        else:
            if(chars not in dictMeters[newLine[0]]):
                dictMeters[newLine[0]]+=[chars]

In [87]:
g.get_postag_dict()[1]

{'VBG': ['summoning',
  'wrinkling',
  'scolding',
  'flatting',
  'scattering',
  'centurieslasting',
  'reassuring',
  'stroking',
  'overseeing',
  'threading',
  'draining',
  'provoking',
  'judging',
  'pling',
  'traveling',
  'welcoming',
  'indulging',
  'accentuating',
  'wedging',
  'shedding',
  'signing',
  'forgiving',
  'running',
  'blustering',
  'lurking',
  'sauntering',
  'sketching',
  'filling',
  'sobbing',
  'whipping',
  'unloading',
  'rinsing',
  'straightening',
  'ing',
  'greeing',
  'gorging',
  'discouraging',
  'using',
  'operating',
  'clearing',
  'lulling',
  'pacing',
  'trapping',
  'kicking',
  'confronting',
  'chopping',
  'banging',
  'wandering',
  'casting',
  'sparkling',
  'yelling',
  'checking',
  'dissecting',
  'glancing',
  'succeeding',
  'loitering',
  'enveloping',
  'thinning',
  'coachmaking',
  'wedding',
  'etching',
  'purifying',
  'labouring',
  'accumulating',
  'turning',
  'heaping',
  'caressing',
  'describing',
  'pump

In [14]:
pos_syllables = {}

for k, v in g.get_postag_dict()[1].items():
    pos_syllables[k] = set()
    for w in v:
        try:
            pos_syllables[k].add(len(dictMeters[w][0]))
        except:
            continue
pos_syllables[','].add(0)
pos_syllables['.'].add(0)

In [15]:
def get_all_partitions(n, I=1):
    yield (n,)
    for i in range(I, n//2 + 1):
        for p in get_all_partitions(n-i, i):
            yield (i,) + p

In [16]:
def get_all_partition_size_n(n, partition_size):
    return [p for p in get_all_partitions(n) if len(p) == partition_size]    

# Checks if candidate syllable partition fits template
def valid_syll(sylls, template, pos_syllables):
    for i in range(len(template)):
        # Add in zeros to account for punctuation
        if template[i] == ',' or template[i] == '.':
            sylls.insert(i, 0)
        if sylls[i] not in pos_syllables[template[i]]:
            return False
    return True

def valid_permutation_sylls(partitions, template, pos_syllables, last_word_sylls):
    for partition in partitions:
        # Goes through all permutations by index, not numbers,
        # inefficient implementation
        for perm in permutations(partition):
            perm = list(perm)
            # Last word is fixed
            perm.append(last_word_sylls)
            if valid_syll(perm, template, pos_syllables):
                return perm

In [75]:
def gen_line(w1, pos_syllables, dictMeters, template=None, temp_len=10, num_sylls=10):
    last_word_sylls = len(dictMeters[w1][0])
    if template is None:
        w1_pos = g.get_postag_dict()[2][w1][0]
        template = []
        # Assumes template of length temp_len exists
        while len(template) <= num_sylls - 4 or len(template) > num_sylls - last_word_sylls + 1:
            template = np.random.choice(templates_dict[w1_pos], 1).item()
        
    # Assign syllables to each pos in template
    # Contains duplicates, inefficient
    num_zero_sylls = sum(1 if pos == '.' or pos == ',' else 0 for pos in template)
    all_partitions = get_all_partition_size_n(num_sylls - last_word_sylls,
                                              len(template) - num_zero_sylls - 1)
    template_sylls = valid_permutation_sylls(all_partitions, template, pos_syllables, last_word_sylls)
    
#     print(num_sylls - last_word_sylls)
#     print(len(template) - num_zero_sylls - 1)
    
    if template_sylls is None:
        raise ValueError('Cannot construct valid meter using template')
    
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            state = sess.run(model.initial_state)
            init_score = np.array([[0]])

            lst = search_back(model, vocab, init_score,[w1],state, sess, 1,
                          g.get_postag_dict()[2], g.get_dict_pos(),g.get_width(),
                              g.get_wordPools()[wordPool_ind], g.get_pos_set(), template, 
                              template_sylls, dictMeters)
            lst.sort(key=itemgetter(0), reverse = True)
        else:
            raise IOError('No checkpoint')
    return template, lst

In [18]:
templates_dict = {}

for l in g.get_postag_dict()[0].values():
    for t, _ in l:
        ending_pos = t[-1]
        if ending_pos not in templates_dict:
            templates_dict[ending_pos] = []
        templates_dict[ending_pos].append(t)

In [19]:
def template_generalization(templates_dict, words, n_words=10):
    scored_dict = {}
    for pos, templates in templates_dict.items():
        scored_dict[pos] = []
        # random words with pos w/o replacement
        if len(words[pos]) < n_words:
            continue
        words = np.random.choice(words[pos], n_words)
        for t in templates:
            template_score = 0
            for w in words:
                print(w, t)
                _, out = gen_line(w, template = t)
                template_score += out[0][0].item() / len(t)
            scored_dict[pos].append((template_score / n_words, t))
    return scored_dict

In [58]:
def gen_poem_first_line(w):
    first_lines = [['there', 'once', 'was', 'a', 'kind', 'old'],
                  ['there', 'was', 'a', 'man', 'named'],
                  ['there', 'was', 'a', 'woman', 'named'],
                  ['there', 'was', 'a']]
    
    first_templates = [['EX', 'RB', 'VBD', 'DT', 'JJ', 'JJ'],
                       ['EX', 'VBD', 'DT', 'NN', 'VBN'],
                       ['EX', 'VBD', 'DT', 'NN', 'VBN'],
                      ['EX', 'VBD', 'DT']]
                       
    for i in range(len(first_lines)):
        first_lines[i].append(w)
        first_templates[i].append(g.get_postag_dict()[2][w][0])
    
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            
            ret = []
            for i in range(len(first_lines)):
                prob, state = model.score_a_list_new(sess, vocab, first_lines[i])
                ret.append((prob / len(first_lines[i]), first_lines[i], first_templates[i]))
            ret.sort(key=lambda x: x[0], reverse=True)
    return ret

In [76]:
def gen_poem(w, pos_syllables, dictMeters):
    five_words = get_five_words(w, g.get_postag_dict()[1])
    
    score, first_line, first_template = gen_poem_first_line(five_words[0])[0]
    template_lengths = [len(first_template), 5, 5, len(first_template) + 1]
    
    first_line_sylls = sum(len(dictMeters[w][0]) for w in first_line)
    syll_lengths = [first_line_sylls, 6, 6, first_line_sylls + 1]
    
    lines = []
    lines.append((first_line, score, first_template))
    for i, w in enumerate(five_words):
        if i == 0:
            continue
        t, out = gen_line(w, pos_syllables, dictMeters, temp_len=template_lengths[i - 1], 
                          num_sylls=syll_lengths[i - 1])
    
        this_line = out[0][1][1]
        this_score = out[0][0].item() / len(this_line)
        lines.append((this_line, this_score, t))
    return lines

In [72]:
get_five_words('war', g.get_postag_dict()[1])

('centaur', 'war', 'extend', 'end', 'for')

In [74]:
%%time

lines = gen_poem('war', pos_syllables, dictMeters)

INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
['there', 'was', 'a', 'man', 'named', 'centaur']
['EX', 'VBD', 'DT', 'NN', 'VBN', 'NN']
NN
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
VB
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
NN
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
IN
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
CPU times: user 54.5 s, sys: 14.5 s, total: 1min 8s
Wall time: 58.4 s


In [77]:
for l, s, t in lines:
    print('{:50} line score: {:2.3f}'.format(' '.join(l), s))
    print(t)

there was a man named centaur                      line score: -7.341
['EX', 'VBD', 'DT', 'NN', 'VBN', 'NN']
on the right hand into war                         line score: -3.339
['IN', 'DT', 'NN', 'NN', 'IN', 'NN']
part of the world extend                           line score: -3.002
['NN', 'IN', 'DT', 'NN', 'VB']
but a few twenty end                               line score: -2.907
['CC', 'DT', 'JJ', 'NN', 'NN']
my right hand while i cant wait for                line score: -3.579
['PRP$', 'NN', 'NN', 'NN', 'PRP', 'VBP', 'VB', 'IN']


In [78]:
%%time

lines = gen_poem('sat', pos_syllables, dictMeters)

INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
CPU times: user 47.1 s, sys: 15.2 s, total: 1min 2s
Wall time: 51.8 s


In [79]:
for l, s, t in lines:
    print('{:50} line score: {:2.3f}'.format(' '.join(l), s))
    print(t)

there was a man named platte                       line score: -7.258
['EX', 'VBD', 'DT', 'NN', 'VBN', 'NNP']
eyes with deep breath and sat                      line score: -3.464
['NNS', 'IN', 'JJ', 'NN', 'CC', 'VBD']
part of the world admit                            line score: -2.925
['NN', 'IN', 'DT', 'NN', 'VB']
the left feet that lets sit                        line score: -4.254
['DT', 'VBN', 'NNS', 'WDT', 'NNS', 'VBP']
it was a man that lost cat                         line score: -3.558
['PRP', 'VBD', 'DT', 'NN', 'WDT', 'VBD', 'NN']


In [45]:
lim_templates = []
with open('data/limericks.txt') as f:
    for line in f:
        line = line.strip()
        tag = nltk.pos_tag(line.split(' '))
        l, t = list(zip(*tag))
        print(l)
        print(t)
        lim_templates.append(t)

('A', 'fellow', 'jumped', 'off', 'a', 'high', 'wall')
('DT', 'NN', 'VBD', 'RP', 'DT', 'JJ', 'NN')
('And', 'had', 'a', 'most', 'terrible', 'fall.')
('CC', 'VBD', 'DT', 'RBS', 'JJ', 'NN')
('He', 'went', 'back', 'to', 'bed')
('PRP', 'VBD', 'RB', 'TO', 'NN')
('With', 'a', 'bump', 'on', 'his', 'head')
('IN', 'DT', 'NN', 'IN', 'PRP$', 'NN')
("That's", 'why', 'you', "don't", 'jump', 'off', 'a', 'wall')
('NNP', 'WRB', 'PRP', 'VBP', 'VB', 'RP', 'DT', 'NN')


In [46]:
def gen_poem_templates(w1, pos_syllables, dictMeters, templates):
    five_words = get_five_words(w1, g.get_postag_dict()[1])

#     score, first_line, first_template = gen_poem_first_line(five_words[0])[0]
    template_lengths = [len(t) for t in templates]

    first_sylls = template_lengths[0] + 1
#     first_line_sylls = sum(len(dictMeters[w][0]) for w in first_line)
#     syll_lengths = [first_line_sylls, 6, 6, first_line_sylls + 1]

    lines = []
    for i, w in enumerate(five_words):
        sylls = template_lengths[i] + len(dictMeters[w][0]) + random.randint(0,1) - 1
        t, out = gen_line(w, pos_syllables, dictMeters, template=templates[i],
                          temp_len=template_lengths[i], num_sylls=sylls)

        
        this_line = out[0][1][1]
        this_score = out[0][0].item() / len(this_line)
        lines.append((this_line, this_score, t))
    return lines

In [54]:
lines = gen_poem_templates('tall', pos_syllables, dictMeters, lim_templates)

INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959


In [55]:
for l, s, t in lines:
    print('{:50} line score: {:2.3f}'.format(' '.join(l), s))
    print(t)

the man stepped down the open mall                 line score: -3.120
('DT', 'NN', 'VBD', 'RP', 'DT', 'JJ', 'NN')
and saw the most dark tall                         line score: -3.841
('CC', 'VBD', 'DT', 'RBS', 'JJ', 'NN')
it was not to appreciate                           line score: -2.255
('PRP', 'VBD', 'RB', 'TO', 'NN')
in the sight of our great                          line score: -2.608
('IN', 'DT', 'NN', 'IN', 'PRP$', 'NN')
come where i cant pick up either all               line score: -4.449
('NNP', 'WRB', 'PRP', 'VBP', 'VB', 'RP', 'DT', 'NN')


In [72]:
from Limericks import Limerick_Generate

In [73]:
lg = Limerick_Generate()

In [74]:
lg.get_five_words('sat')

('democrat', 'sat', 'admit', 'sit', 'cat')